In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import numpy as np
import time
from game import Game
from agents import ExpectiMaxAgent
from expectimax import board_to_move
from my2048 import one_hot_encode, Net

print("A")

BATCH_SIZE = 128
EPOCH = 10
BATCH = 500
THRESHOLD = 128
MAX_SCORE = 256

DATA = torch.zeros(BATCH, BATCH_SIZE, 16, 4, 4)
LABELS = torch.zeros(BATCH, BATCH_SIZE, dtype=torch.long)
game = Game(size=4, score_to_win=MAX_SCORE, random=False)
agent = ExpectiMaxAgent(game)
while game.score<THRESHOLD:
    agent.game.move(board_to_move(agent.game.board))
data = torch.zeros(BATCH_SIZE, 16, 4, 4)
labels = torch.zeros(BATCH_SIZE, dtype=torch.long)

for j in range(BATCH):
    for i in range(BATCH_SIZE):
        if agent.game.end:
            game = Game(size=4, score_to_win=MAX_SCORE, random=False)
            agent = ExpectiMaxAgent(game)
            while game.score<THRESHOLD:
                agent.game.move(board_to_move(agent.game.board))
        data[i, :] = one_hot_encode(agent.game.board)
        labels[i] = board_to_move(agent.game.board)
        agent.game.move(labels[i])
    print('BATCH %d/%d 準備完了です'%(j + 1, BATCH))

    DATA[j] = data
    LABELS[j] = labels

In [ ]:
model = Net()
PATH = './2048-%d.pth'%THRESHOLD
model.load_state_dict(torch.load(PATH))

if torch.cuda.is_available():
    print("gpu is available")
    model.cuda()
    DATA = DATA.cuda()
    LABELS = LABELS.cuda()
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.002)
model.train()

for j in range(EPOCH):
    correct = 0
    for i in range(BATCH):
        data = DATA[i]
        labels = LABELS[i]
        optimizer.zero_grad()
        outputs = model(data)
        _, predict = torch.max(outputs.data, -1)
        correct += int(sum(predict == labels))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        #print('loss:%.3f' % (loss.item()))
    print('accuracy:%.3f%%' % (correct / BATCH_SIZE / BATCH * 100))
        #time.sleep(1)
    print("EPOCH %d OK!"%(j+1))
    
model.cpu()
torch.save(model.state_dict(), PATH)

print("完了します")